In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import random
from PIL import Image
from IPython.core.display import HTML

In [ ]:
# Define the parameters
num_points = 100
from_x = -1
to_x = 1

noise_factor = 0.5

grade = 2

# Generate random coefficients for the polynomial
coefficients = [random.uniform(0, 10) for i in range(grade+1)]

# Generate the X values
X = np.linspace(from_x, to_x, num_points)

# Generate the Y values for the polynomial
Y = np.zeros_like(X)
for i, coef in enumerate(coefficients):
    Y += coef * np.power(X, i)

# Add noise to the Y values
Y += np.random.normal(from_x, noise_factor, num_points)


#mean_y = np.mean(Y)
#std_y = np.std(Y)

# Normalize the Y values
#Y = (Y - mean_y ) / std_y

# Normalize the Y coefficient
# coefficients = [c / np.power(std_y, i+1) for i, c in enumerate(coefficients)
#print(f"> Polynomial coefficients: {coefficients}")

# Plot the data
plt.scatter(X, Y)
plt.show()

In [ ]:
def f_error(n, THETA, X, Y):
    
    # Extract the theta values from THETA array
    theta_0 = THETA[0]
    
    theta_rest = THETA[1:]
    
    # Generate the polynomial using theta_rest values
    poly = sum(theta_rest[i] * np.power(X, i+1) for i in range(len(theta_rest)))
    
    # Calculate the mean squared error
    mse = (1/n) * np.sum((Y - (theta_0 + poly))**2)
    
    return mse


def get_thetaj_derivative(n, X, j, Y, Y_pred):
    return (-2 / n) * sum( (X**j) * (Y - Y_pred)) 

#def get_theta0_derivative(n, Y, Y_pred):
#    return (-2 / n) * sum(Y - Y_pred)

In [ ]:
#-------------------
# Iper parametri
#-------------------

dynamic_lr = True

# learning Rate
lr = 0.01

if dynamic_lr:
    lr = 0

# Numero di epoche
epochs = 500 


#-------------------
# Parametri
#-------------------

first_theta = 0

if dynamic_lr:
    first_theta = 1
    
    
THETA = [first_theta]*(grade + 1)

In [ ]:
# numero di esempi del dataset
n = len(X)  

# array che conterrà i vari valori assunti dal parametro m dopo ogni epoca
THETA_history = []

# array che conterrà i vari valori assunti dall'errore dopo ogni epoca
errors = []

print ("Start fitting process")
print("---")

#-------------------------
# Discesa del Gradiente
#----------------------
for i in range(epochs):
    
    err = f_error(n, THETA, X, Y)
    
    if dynamic_lr:
        norm = np.linalg.norm(THETA)

    print(f"> Epoch {i} | Err: {err:.5f}")
    
    # Append sugli array M e errors
    errors.append(err)

    THETA_history.append(THETA.copy())

    # Extract the theta values from THETA array
    theta_0 = THETA[0]
    theta_rest = THETA[1:]
    poly = sum(theta_rest[j] * np.power(X, j+1) for j in range(len(theta_rest)))
    
    #predizioni di y
    Y_pred = theta_0 + poly    
    
    
    if dynamic_lr:
        
        if norm >= 1:
            lr = 1/norm
            
        else:
            lr = norm
    
    for j in range( len(THETA) ):
        THETA[j] -= lr * get_thetaj_derivative(n, X, j, Y, Y_pred)
    

print("---")
print ("Fit completed")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse

data = {7.865058230145619: 24.65887271753253,
        8.345097456250649: 42.029308148619485,
        8.825136682355678: 94.13996635342133}

HSVColorConverter(inputString="blue sky").getRGB()
HSVColorConverter().getRGB(intensity=1) 

# Convert data into numpy arrays
x_vals = np.array([key for key, val in data.items()])
y_vals = np.asarray(list(map(lambda x : x[1] ,data.items())))

# Create empty array for color map
color = []

# Loop through each value and assign a random RGB color based on intensity
for i in range(len(y_vals)):
    hue = colorConverter.to_rgb(intensity=y_vals[i], colorSpace='hsv')
    hex_code = ''.join([f"{channel:02X}" for channel in hue[:3]])
    color.append("#"+hex_code)
    
scatter = ax.scatter(x_vals, y_vals, c=np.random.choice(colors), marker="o", edgecolors="none")
ax.set_title("Scatter Plot with Color Map")
ax.set_xlabel('Key')
ax.set_ylabel('Value')
plt.show()

In [ ]:
# Extract the theta values from THETA array
theta_0 = THETA[0]
theta_rest = THETA[1:]
poly = sum(theta_rest[j] * np.power(X, j+1) for j in range(len(theta_rest)))
# predizioni di y
Y_pred = theta_0 + poly


fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Plot of real data and predicted regression line
axs[0].scatter(X, Y)
axs[0].plot(X, Y_pred, color="red")

axs[0].set_xlabel("X")
axs[0].set_ylabel("Y")
axs[0].set_title("Prediction over Real Data")

# Plot of error vs epoches
axs[1].plot(errors)
axs[1].set_xlabel("Epoches")
axs[1].set_ylabel("Error")
axs[1].set_title(f'Errors | Max Err {max(errors):.3e} | Min Err {min(errors):.3e}')

plt.tight_layout()
plt.show()

In [ ]:
gif_filename = "polynomial_target_approach.gif"

# Initialize figure
fig, ax = plt.subplots()

# Set axis labels and title
ax.set_xlabel('x')
ax.set_ylabel('y')


# Initialize empty list for frames
frames = []


print('Starting creating the gif frames')
print('---')

# Create the scatter plot for each point
for i in range(epochs):
    
    t = THETA_history[i]
    
    # Compute the predicted y values using the current THETA values
    
    y_pred = np.zeros_like(X)
    
    for j in range( len(t) ):
        y_pred += t[j] * (X**j) 

    ax.scatter(X, Y)
    ax.plot(X, y_pred, color="red")

    #ax.set_title(f"E:{i:d} m:{M[i]:.3e} c:{C[i]:.3e} Err: {errors[i]}")
   
    # Save the current plot as a PIL image
    fig.canvas.draw()
    frame = Image.frombytes('RGB', fig.canvas.get_width_height(), fig.canvas.tostring_rgb())
    frames.append(frame)

    # Clear the plot for the next iteration
    ax.clear()

    # set the tick density on the X and Y axes
    ax.locator_params(axis='x', nbins=10)
    ax.locator_params(axis='y', nbins=10)


    
    ax.set_title(f"Epoch:{i:d} | Err: {errors[i]:.3e}")
    print(f"Epoch:{i:d} | Err: {errors[i]:.3e}")
    
    
    # Close the plot to avoid displaying it
    plt.close()

print('---')
print('saving the gif (it can take some time, especially with many epoches)')
print('Please waiting...')


# Save the gif
frames[0].save(gif_filename, format='GIF', append_images=frames[1:], save_all=True, duration=10, loop=0)
print('The gif has been successfully created!')
HTML(f'To open it <a href="{gif_filename}" target="_blank"> click here</a>')